In [ ]:
import json
import boto3
import requests
from datetime import datetime
import os

s3_client = boto3.client('s3')

# Variables de entorno (configuradas en Lambda)
API_URL = os.environ['API_URL']
API_KEY = os.environ['API_KEY']
DESTINATION_BUCKET = os.environ['DESTINATION_BUCKET']
RAW_PREFIX = "raw-data/"

def lambda_handler(event, context):

    try:
        # Llamada a API externa
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        }

        response = requests.get(API_URL, headers=headers, timeout=30)

        if response.status_code != 200:
            raise Exception(f"Error API: {response.status_code} - {response.text}")

        data = response.json()

        # Generar nombre único
        timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
        file_key = f"{RAW_PREFIX}data_{timestamp}.json"

        # Guardar en S3 (RAW sin modificar)
        s3_client.put_object(
            Bucket=DESTINATION_BUCKET,
            Key=file_key,
            Body=json.dumps(data),
            ContentType='application/json'
        )

        return {
            "statusCode": 200,
            "body": f"Archivo guardado en {file_key}"
        }

    except Exception as e:
        print(f"Error en ingesta: {str(e)}")
        raise e
